***Conversational AI - Chatbot!***

***Import Libraries***

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

***Environment setup***

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'

In [ ]:
system_message = "You are a helpful assistant"

This code defines a Python function called chat that takes two arguments: ***message and history***. This function is designed to handle a turn in a chat conversation.

Here's a breakdown:

def chat(message, history):: This line defines the function named chat and indicates that it accepts two parameters:
message: This is likely the current message from the user.
history: This is expected to be a list representing the previous turns in the conversation.
messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]: This line constructs the complete list of messages to be sent to the language model.
[{"role": "system", "content": system_message}]: It starts with a system message. The content of this message comes from the system_message variable, which is expected to be defined elsewhere and provides instructions or context to the language model.
+ history: It then adds the existing history (the previous conversational turns) to the list.
+ [{"role": "user", "content": message}]: Finally, it appends the current user message to the list, formatted as a user message.
print("History is:") and print(history): These lines print the content of the history variable to the console. This is likely for debugging purposes, allowing you to see the conversation history being passed to the function.
print("And messages is:") and print(messages): These lines print the complete list of messages that will be sent to the language model, also likely for debugging.
stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True): This line makes a streaming API call to a language model using the openai library.
model=MODEL: Specifies the language model to use. The model name is taken from the MODEL variable, which is expected to be defined elsewhere.
messages=messages: Provides the complete list of messages constructed earlier as the conversation history.
stream=True: Enables streaming of the response, meaning the response will be received in chunks.
response = "": Initializes an empty string variable response to accumulate the streamed content.
for chunk in stream:: This loop iterates over the chunks of the response as they are streamed from the API.
response += chunk.choices[0].delta.content or '': Inside the loop, it appends the content of each chunk to the response string, handling potential None values.

yield response: This is a key part of the function. yield makes this function a generator. Instead of returning the final response all at once, it yields the accumulating response string in each iteration of the loop. This allows the caller of the function to receive and process the response as it's being generated, which is useful for displaying streaming output in real-time.
In essence, the chat function takes a user message and the conversation history, constructs the full message list for the language model (including a system message), makes a streaming API call to the model, and yields the accumulating response as it is streamed back. This function is likely designed to be used in an interactive chat interface where you want to display the model's response as it's being generated.

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

***UI Setup***

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()